In [1]:
from skimage.filters import gabor_kernel
from skimage import io
from skimage.transform import resize
from matplotlib import pyplot as plt 
import numpy as np
import tensorflow_datasets as tfds
import tensorflow as tf

In [5]:
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical
import math

AUTOTUNE = tf.data.experimental.AUTOTUNE
BATCH_SIZE = 100
IMAGE_SIZE = 32
NUM_OF_CLASSES = 10

# The data, split between train and test sets:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
# x_train = x_train.astype('float32') / 255.0
# x_test = x_test.astype('float32') / 255.0

y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)

AUTOTUNE = tf.data.experimental.AUTOTUNE
BATCH_SIZE = 100
IMAGE_SIZE = 32
NUM_OF_CLASSES = 10

def process_dataset(image, label):
    # Normalize images to have a mean of 0 and standard deviation of 1
#     image = tf.image.per_image_standardization(image)
    
#     image = tf.image.convert_image_dtype(image, tf.float32)
    # Resize images from 32x32 to 227x227
#     image = tf.image.resize(image, (IMAGE_SIZE,IMAGE_SIZE))

#     image = image/255.0
         
#     label = tf.one_hot(tf.cast(label, tf.int32), NUM_OF_CLASSES)
        
    return image, label

train_data = tf.data.Dataset.from_tensor_slices((x_train, y_train))
test_data = tf.data.Dataset.from_tensor_slices((x_test, y_test))

def construct_dataset(ds):
    ds = ds.shuffle(buffer_size=BATCH_SIZE)
    ds = ds.map(process_dataset,num_parallel_calls=AUTOTUNE)
#     ds=ds.repeat()
    ds = ds.batch(BATCH_SIZE, drop_remainder=False)
    ds = ds.prefetch(buffer_size=AUTOTUNE)
    return ds

train_data = construct_dataset(train_data)
test_data = construct_dataset(test_data)

steps_per_epoch=math.ceil(50000/BATCH_SIZE)
validation_steps=math.ceil(10000/BATCH_SIZE)


x_train shape: (50000, 32, 32, 3)
x_test shape: (10000, 32, 32, 3)


In [6]:
# Importing the Keras libraries and packages
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Activation
from tensorflow.keras.layers import Input, Dense, Dropout, BatchNormalization
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import RMSprop, Adam

# dimensionality of input and latent encoded representations
inpt_dim = (32, 32, 3)

inpt_img = Input(shape=inpt_dim)

# rescale = tf.keras.layers.experimental.preprocessing.Rescaling(1./255, input_shape = inpt_dim )(inpt_img)

# Block 1
cl1 = Conv2D(64, (3, 3), strides=(2, 2),activation='relu', input_shape = inpt_dim)(inpt_img)
bnl2 = BatchNormalization()(cl1)
# afl3 = Activation('relu')(bnl2)
pl4 = MaxPooling2D(pool_size = (2, 2))(bnl2)

# Adding a second convolutional layer
cl5 = Conv2D(128, (3, 3), strides=(1, 1), activation='relu')(pl4)
bnl6 = BatchNormalization()(cl5)
# afl7 = Activation('relu')(bnl6)
pl8 = MaxPooling2D(pool_size = (2, 2))(bnl6)
bnl9 = BatchNormalization()(pl8)

# Step 3 - Flattening
fl10 = Flatten()(bnl9)

# Step 4 - Full connection
dol11 = Dropout(0.5)(fl10)
dl12 = Dense(units = 256, activation = 'relu')(dol11)
dol13 = Dropout(0.2)(dl12)
dl14 = Dense(units = 64, activation = 'relu')(dol13)
dol15 = Dropout(0.1)(dl14)
output = Dense(units = 10, activation = 'sigmoid')(dol15)

classifier = Model(inpt_img, output)

# Compiling the CNN
opt = RMSprop(learning_rate=0.001)
# opt = Adam(learning_rate=0.01)

classifier.compile(optimizer = opt, loss ='binary_crossentropy', 
                   metrics = ['accuracy'])

print(classifier.summary())

Model: "functional_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 32, 32, 3)]       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 15, 15, 64)        1792      
_________________________________________________________________
batch_normalization_3 (Batch (None, 15, 15, 64)        256       
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 7, 7, 64)          0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 5, 5, 128)         73856     
_________________________________________________________________
batch_normalization_4 (Batch (None, 5, 5, 128)         512       
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 2, 2, 128)        

In [ ]:
# Fitting the CNN to the images

from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping

reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=10,  
                              min_delta=1e-4, mode='min', verbose=1)

stop_alg = EarlyStopping(monitor='val_loss', patience=35, 
                         restore_best_weights=True, verbose=1)



hist = classifier.fit(train_data,  epochs=1000, 
                   callbacks=[stop_alg, reduce_lr], 
                      validation_steps=validation_steps,
                      steps_per_epoch=steps_per_epoch,
                   validation_data=test_data)



Epoch 1/1000
500/500 [==============================] - 37s 74ms/step - loss: 0.2659 - accuracy: 0.3861 - val_loss: 0.2258 - val_accuracy: 0.4841
Epoch 2/1000
500/500 [==============================] - 37s 73ms/step - loss: 0.2204 - accuracy: 0.5047 - val_loss: 0.2394 - val_accuracy: 0.4426
Epoch 3/1000
500/500 [==============================] - 38s 75ms/step - loss: 0.2032 - accuracy: 0.5486 - val_loss: 0.1916 - val_accuracy: 0.5782
Epoch 4/1000
500/500 [==============================] - 37s 74ms/step - loss: 0.1937 - accuracy: 0.5765 - val_loss: 0.1962 - val_accuracy: 0.5560
Epoch 5/1000
500/500 [==============================] - 37s 74ms/step - loss: 0.1860 - accuracy: 0.5935 - val_loss: 0.1886 - val_accuracy: 0.5797
Epoch 6/1000
500/500 [==============================] - 38s 75ms/step - loss: 0.1797 - accuracy: 0.6110 - val_loss: 0.1887 - val_accuracy: 0.5888
Epoch 7/1000
500/500 [==============================] - 45s 89ms/step - loss: 0.1746 - accuracy: 0.6241 - val_loss: 0.1711 -

500/500 [==============================] - 60s 120ms/step - loss: 0.1077 - accuracy: 0.7824 - val_loss: 0.1347 - val_accuracy: 0.7226
Epoch 56/1000
500/500 [==============================] - 61s 123ms/step - loss: 0.1071 - accuracy: 0.7822 - val_loss: 0.1415 - val_accuracy: 0.7014
Epoch 57/1000
500/500 [==============================] - 62s 124ms/step - loss: 0.1083 - accuracy: 0.7784 - val_loss: 0.1440 - val_accuracy: 0.7009
Epoch 58/1000
500/500 [==============================] - 65s 131ms/step - loss: 0.1073 - accuracy: 0.7829 - val_loss: 0.1414 - val_accuracy: 0.7014
Epoch 59/1000
500/500 [==============================] - 63s 127ms/step - loss: 0.1070 - accuracy: 0.7835 - val_loss: 0.1366 - val_accuracy: 0.7195
Epoch 60/1000
500/500 [==============================] - 64s 127ms/step - loss: 0.1071 - accuracy: 0.7842 - val_loss: 0.1413 - val_accuracy: 0.7002
Epoch 61/1000
500/500 [==============================] - 59s 119ms/step - loss: 0.1065 - accuracy: 0.7872 - val_loss: 0.1421 -

500/500 [==============================] - 60s 120ms/step - loss: 0.0912 - accuracy: 0.8187 - val_loss: 0.1252 - val_accuracy: 0.7486
Epoch 108/1000
500/500 [==============================] - 62s 123ms/step - loss: 0.0915 - accuracy: 0.8188 - val_loss: 0.1260 - val_accuracy: 0.7440
Epoch 109/1000
500/500 [==============================] - ETA: 0s - loss: 0.0914 - accuracy: 0.8185
Epoch 00109: ReduceLROnPlateau reducing learning rate to 3.125000148429535e-05.
500/500 [==============================] - 61s 123ms/step - loss: 0.0914 - accuracy: 0.8185 - val_loss: 0.1251 - val_accuracy: 0.7494
Epoch 110/1000
500/500 [==============================] - 61s 122ms/step - loss: 0.0911 - accuracy: 0.8187 - val_loss: 0.1248 - val_accuracy: 0.7489
Epoch 111/1000
500/500 [==============================] - 62s 124ms/step - loss: 0.0911 - accuracy: 0.8181 - val_loss: 0.1247 - val_accuracy: 0.7506
Epoch 112/1000
500/500 [==============================] - 63s 125ms/step - loss: 0.0919 - accuracy: 0.815

500/500 [==============================] - 47s 94ms/step - loss: 0.0886 - accuracy: 0.8235 - val_loss: 0.1241 - val_accuracy: 0.7501
Epoch 157/1000
500/500 [==============================] - 46s 92ms/step - loss: 0.0900 - accuracy: 0.8231 - val_loss: 0.1241 - val_accuracy: 0.7498
Epoch 158/1000
500/500 [==============================] - 46s 93ms/step - loss: 0.0894 - accuracy: 0.8225 - val_loss: 0.1240 - val_accuracy: 0.7498
Epoch 159/1000
500/500 [==============================] - 46s 92ms/step - loss: 0.0893 - accuracy: 0.8225 - val_loss: 0.1240 - val_accuracy: 0.7493
Epoch 160/1000
500/500 [==============================] - 47s 95ms/step - loss: 0.0905 - accuracy: 0.8186 - val_loss: 0.1240 - val_accuracy: 0.7503
Epoch 161/1000
500/500 [==============================] - 46s 92ms/step - loss: 0.0890 - accuracy: 0.8238 - val_loss: 0.1240 - val_accuracy: 0.7500
Epoch 162/1000
248/500 [=============>................] - ETA: 22s - loss: 0.0903 - accuracy: 0.8233